In [33]:
%matplotlib inline
import os
import itertools
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Range1d

from climatools.fortran.read import numbers

from IPython import display

In [34]:
output_notebook()

Loading BokehJS ...

In [35]:
'''
Atmosphere profile data.  These are literally the contents of 
mls75_r8.pro, saw75_r8.pro and trp75_r8 copy-n-pasted.
'''

def s_mls75(var):
    if var == 'p':
        return '''
      data (plevel(i),i=1,nlayer+1)/                                                      
     &      0.0000000_r8, 0.0006244_r8, 0.0008759_r8, 0.0012286_r8,                       
     &      0.0017234_r8,                                                                 
     &      0.0024174_r8, 0.0033909_r8, 0.0047565_r8, 0.0066720_r8,                       
     &      0.0093589_r8,                                                                 
     &      0.0131278_r8, 0.0184145_r8, 0.0258302_r8, 0.0362323_r8,                       
     &      0.0508234_r8,                                                                 
     &      0.0712906_r8, 0.1000000_r8, 0.1402710_r8, 0.1967600_r8,                       
     &      0.2759970_r8,                                                                 
     &      0.3871_r8, 0.5431_r8, 0.7617_r8, 1.0685_r8, 1.4988_r8,                        
     &      2.1024_r8,                                                                    
     &      2.9490_r8, 4.1366_r8, 5.8025_r8, 8.1392_r8, 11.4170_r8,                       
     &      16.0147_r8,                                                                   
     &      22.4640_r8, 31.5105_r8, 44.2001_r8, 62.0000_r8, 85.7750_r8,                   
     &      109.5500_r8,                                                                  
     &      133.3250_r8, 157.1000_r8, 180.8750_r8, 204.6500_r8,                           
     &      228.4250_r8, 252.2000_r8,                                                     
     &      275.9750_r8, 299.7500_r8, 323.5250_r8, 347.3000_r8,                           
     &      371.0750_r8, 394.8500_r8,                                                     
     &      418.6250_r8, 442.4000_r8, 466.1750_r8, 489.9500_r8,                           
     &      513.7250_r8, 537.5000_r8,                                                     
     &      561.2750_r8, 585.0500_r8, 608.8250_r8, 632.6000_r8,                           
     &      656.3750_r8, 680.1500_r8,                                                     
     &      703.9250_r8, 727.7000_r8, 751.4750_r8, 775.2500_r8,                           
     &      799.0250_r8, 822.8000_r8,                                                     
     &      846.5750_r8, 870.3500_r8, 894.1250_r8, 917.9000_r8,                           
     &      941.6750_r8, 965.4500_r8,                                                     
     &      989.2250_r8, 1013.0000_r8/
        '''
    elif var == 't':
        return '''
      data (tlayer(i),i=1,nlayer)/                                                        
     &      209.9704_r8, 210.2097_r8, 210.7183_r8, 211.2577_r8,                           
     &      211.7986_r8, 212.3405_r8,                                                     
     &      212.8834_r8, 213.4273_r8, 213.9723_r8, 214.5183_r8,                           
     &      215.0652_r8, 215.6132_r8,                                                     
     &      216.1626_r8, 216.7491_r8, 218.4804_r8, 223.1608_r8,                           
     &      229.9159_r8, 237.0121_r8,                                                     
     &      244.3231_r8, 251.8528_r8, 259.6010_r8, 267.5217_r8,                           
     &      273.8894_r8, 274.3845_r8,                                                     
     &      269.4364_r8, 263.1005_r8, 256.6132_r8, 250.3104_r8,                           
     &      244.4832_r8, 239.1006_r8,                                                     
     &      233.8843_r8, 228.8750_r8, 224.8026_r8, 221.8245_r8,                           
     &      219.1726_r8, 216.8415_r8,                                                     
     &      215.7880_r8, 215.7518_r8, 215.7807_r8, 216.4645_r8,                           
     &      219.2781_r8, 223.7674_r8,                                                     
     &      228.2605_r8, 232.4265_r8, 236.2986_r8, 239.9184_r8,                           
     &      243.3190_r8, 246.5257_r8,                                                     
     &      249.5573_r8, 252.4255_r8, 255.1358_r8, 257.6929_r8,                           
     &      260.1064_r8, 262.3929_r8,                                                     
     &      264.5716_r8, 266.6589_r8, 268.6667_r8, 270.6037_r8,                           
     &      272.4764_r8, 274.2898_r8,                                                     
     &      276.0482_r8, 277.7552_r8, 279.4135_r8, 281.0244_r8,                           
     &      282.5865_r8, 284.0929_r8,                                                     
     &      285.5257_r8, 286.8554_r8, 288.0568_r8, 289.1321_r8,                           
     &      290.1115_r8, 291.0293_r8,                                                     
     &      291.9081_r8, 292.7597_r8, 293.5899_r8/        
        '''
    elif var == 'h2o':
        return '''
      data (wlayer(i),i=1,nlayer)/                                                 
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4003E-05_r8,            
     &      0.4170E-05_r8,                                                         
     &      0.6396E-05_r8, 0.1702E-04_r8, 0.3855E-04_r8, 0.7691E-04_r8,            
     &      0.1385E-03_r8,                                                         
     &      0.1948E-03_r8, 0.2521E-03_r8, 0.3203E-03_r8, 0.4006E-03_r8,            
     &      0.4940E-03_r8,                                                         
     &      0.6017E-03_r8, 0.7101E-03_r8, 0.8255E-03_r8, 0.9556E-03_r8,            
     &      0.1102E-02_r8,                                                         
     &      0.1266E-02_r8, 0.1450E-02_r8, 0.1736E-02_r8, 0.2070E-02_r8,            
     &      0.2438E-02_r8,                                                         
     &      0.2841E-02_r8, 0.3276E-02_r8, 0.3743E-02_r8, 0.4241E-02_r8,            
     &      0.4767E-02_r8,                                                         
     &      0.5319E-02_r8, 0.5895E-02_r8, 0.6492E-02_r8, 0.7106E-02_r8,            
     &      0.7734E-02_r8,                                                         
     &      0.8373E-02_r8, 0.9021E-02_r8, 0.9676E-02_r8, 0.1034E-01_r8,            
     &      0.1100E-01_r8/
        '''
    elif var == 'o3':
        return '''
      data (olayer(i),i=1,nlayer)/                                                 
     &      0.1793E-06_r8, 0.2011E-06_r8, 0.2446E-06_r8, 0.2884E-06_r8,            
     &      0.3324E-06_r8,                                                         
     &      0.3767E-06_r8, 0.4212E-06_r8, 0.4659E-06_r8, 0.5108E-06_r8,            
     &      0.5559E-06_r8,                                                         
     &      0.6013E-06_r8, 0.6469E-06_r8, 0.6928E-06_r8, 0.7389E-06_r8,            
     &      0.8089E-06_r8,                                                         
     &      0.1003E-05_r8, 0.1306E-05_r8, 0.1628E-05_r8, 0.1968E-05_r8,            
     &      0.2328E-05_r8,                                                         
     &      0.2707E-05_r8, 0.3108E-05_r8, 0.3664E-05_r8, 0.4604E-05_r8,            
     &      0.5945E-05_r8,                                                         
     &      0.7590E-05_r8, 0.8930E-05_r8, 0.9585E-05_r8, 0.9906E-05_r8,            
     &      0.1008E-04_r8,                                                         
     &      0.9725E-05_r8, 0.8573E-05_r8, 0.7141E-05_r8, 0.5804E-05_r8,            
     &      0.4274E-05_r8,                                                         
     &      0.2663E-05_r8, 0.1571E-05_r8, 0.1041E-05_r8, 0.7856E-06_r8,            
     &      0.6023E-06_r8,                                                         
     &      0.4495E-06_r8, 0.3569E-06_r8, 0.3016E-06_r8, 0.2518E-06_r8,            
     &      0.2143E-06_r8,                                                         
     &      0.1932E-06_r8, 0.1762E-06_r8, 0.1599E-06_r8, 0.1467E-06_r8,            
     &      0.1366E-06_r8,                                                         
     &      0.1271E-06_r8, 0.1181E-06_r8, 0.1096E-06_r8, 0.1028E-06_r8,            
     &      0.9749E-07_r8,                                                         
     &      0.9252E-07_r8, 0.8829E-07_r8, 0.8459E-07_r8, 0.8103E-07_r8,            
     &      0.7782E-07_r8,                                                         
     &      0.7491E-07_r8, 0.7206E-07_r8, 0.6943E-07_r8, 0.6707E-07_r8,            
     &      0.6481E-07_r8,                                                         
     &      0.6258E-07_r8, 0.6073E-07_r8, 0.5928E-07_r8, 0.5789E-07_r8,            
     &      0.5651E-07_r8,                                                         
     &      0.5519E-07_r8, 0.5398E-07_r8, 0.5282E-07_r8, 0.5167E-07_r8,            
     &      0.5054E-07_r8/
        '''
    else:
        raise ValueError('Variable {} not available'.format(var))


def s_saw75(var):
    if var == 'p':
        return '''
      data (plevel(i), i=1, nlayer + 1)/                                           
     &      0.0000000_r8, 0.0006244_r8, 0.0008759_r8, 0.0012286_r8,                
     &      0.0017234_r8,                                                          
     &      0.0024174_r8, 0.0033909_r8, 0.0047565_r8, 0.0066720_r8,                
     &      0.0093589_r8,                                                          
     &      0.0131278_r8, 0.0184145_r8, 0.0258302_r8, 0.0362323_r8,                
     &      0.0508234_r8,                                                          
     &      0.0712906_r8, 0.1000000_r8, 0.1402710_r8, 0.1967600_r8,                
     &      0.2759970_r8,                                                          
     &      0.3871_r8, 0.5431_r8, 0.7617_r8, 1.0685_r8, 1.4988_r8,                 
     &      2.1024_r8,                                                             
     &      2.9490_r8, 4.1366_r8, 5.8025_r8, 8.1392_r8, 11.4170_r8,                
     &      16.0147_r8,                                                            
     &      22.4640_r8, 31.5105_r8, 44.2001_r8, 62.0000_r8, 85.7750_r8,            
     &      109.5500_r8,                                                           
     &      133.3250_r8, 157.1000_r8, 180.8750_r8, 204.6500_r8,                    
     &      228.4250_r8, 252.2000_r8,                                              
     &      275.9750_r8, 299.7500_r8, 323.5250_r8, 347.3000_r8,                    
     &      371.0750_r8, 394.8500_r8,                                              
     &      418.6250_r8, 442.4000_r8, 466.1750_r8, 489.9500_r8,                    
     &      513.7250_r8, 537.5000_r8,                                              
     &      561.2750_r8, 585.0500_r8, 608.8250_r8, 632.6000_r8,                    
     &      656.3750_r8, 680.1500_r8,                                              
     &      703.9250_r8, 727.7000_r8, 751.4750_r8, 775.2500_r8,                    
     &      799.0250_r8, 822.8000_r8,                                              
     &      846.5750_r8, 870.3500_r8, 894.1250_r8, 917.9000_r8,                    
     &      941.6750_r8, 965.4500_r8,                                              
     &      989.2250_r8, 1013.0000_r8/
        '''
    elif var == 't':
        return '''
      data (tlayer(i), i = 1, nlayer)/                                             
     &      211.2100_r8, 212.4826_r8, 215.0756_r8, 217.7324_r8,                    
     &      220.4205_r8, 223.1399_r8,                                              
     &      225.8907_r8, 228.6735_r8, 231.4882_r8, 234.3355_r8,                    
     &      237.2156_r8, 240.1287_r8,                                              
     &      243.0619_r8, 245.6824_r8, 247.7211_r8, 249.4930_r8,                    
     &      251.2686_r8, 253.0555_r8,                                              
     &      254.8535_r8, 256.6617_r8, 258.2965_r8, 257.0200_r8,                    
     &      251.8593_r8, 245.6121_r8,                                              
     &      239.5087_r8, 233.5617_r8, 227.8060_r8, 222.9702_r8,                    
     &      219.6087_r8, 217.0275_r8,                                              
     &      214.7087_r8, 212.7874_r8, 212.2661_r8, 213.2143_r8,                    
     &      214.4897_r8, 215.7518_r8,                                              
     &      216.7051_r8, 217.0524_r8, 217.0658_r8, 217.0659_r8,                    
     &      217.0659_r8, 217.0659_r8,                                              
     &      217.0657_r8, 217.0707_r8, 217.2256_r8, 218.4121_r8,                    
     &      220.9656_r8, 223.9609_r8,                                              
     &      226.8439_r8, 229.5838_r8, 232.1905_r8, 234.6726_r8,                    
     &      237.0358_r8, 239.2826_r8,                                              
     &      241.4128_r8, 243.4232_r8, 245.3092_r8, 247.0656_r8,                    
     &      248.6884_r8, 250.1771_r8,                                              
     &      251.5357_r8, 252.7727_r8, 253.9001_r8, 254.9318_r8,                    
     &      255.8807_r8, 256.7554_r8,                                              
     &      257.5546_r8, 258.2550_r8, 258.7947_r8, 259.0820_r8,                    
     &      259.0641_r8, 258.7891_r8,                                              
     &      258.3628_r8, 257.8709_r8, 257.3581_r8/
        '''
    elif var == 'h2o':
        return '''
      data (wlayer(i), i = 1, nlayer)/                                             
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8, 0.4000E-05_r8,            
     &      0.4000E-05_r8,                                                         
     &      0.4000E-05_r8, 0.4000E-05_r8, 0.4043E-05_r8, 0.5616E-05_r8,            
     &      0.7412E-05_r8,                                                         
     &      0.9552E-05_r8, 0.1130E-04_r8, 0.1308E-04_r8, 0.1520E-04_r8,            
     &      0.1767E-04_r8,                                                         
     &      0.2057E-04_r8, 0.2412E-04_r8, 0.2842E-04_r8, 0.4231E-04_r8,            
     &      0.6649E-04_r8,                                                         
     &      0.9861E-04_r8, 0.1392E-03_r8, 0.1881E-03_r8, 0.2448E-03_r8,            
     &      0.3083E-03_r8,                                                         
     &      0.3770E-03_r8, 0.4493E-03_r8, 0.5231E-03_r8, 0.5965E-03_r8,            
     &      0.6676E-03_r8,                                                         
     &      0.7347E-03_r8, 0.7964E-03_r8, 0.8514E-03_r8, 0.8988E-03_r8,            
     &      0.9383E-03_r8,                                                         
     &      0.9694E-03_r8, 0.9922E-03_r8, 0.1007E-02_r8, 0.1013E-02_r8,            
     &      0.1012E-02_r8,                                                         
     &      0.9983E-03_r8, 0.9718E-03_r8, 0.9458E-03_r8, 0.9209E-03_r8,            
     &      0.8970E-03_r8/
        '''
    elif var == 'o3':
        return '''
      data (olayer(i), i = 1, nlayer)/                                             
     &      0.2683E-06_r8, 0.3122E-06_r8, 0.4013E-06_r8, 0.4926E-06_r8,            
     &      0.5860E-06_r8,                                                         
     &      0.6817E-06_r8, 0.7796E-06_r8, 0.8798E-06_r8, 0.9824E-06_r8,            
     &      0.1087E-05_r8,                                                         
     &      0.1195E-05_r8, 0.1305E-05_r8, 0.1417E-05_r8, 0.1664E-05_r8,            
     &      0.2106E-05_r8,                                                         
     &      0.2612E-05_r8, 0.3125E-05_r8, 0.3645E-05_r8, 0.4172E-05_r8,            
     &      0.4706E-05_r8,                                                         
     &      0.5244E-05_r8, 0.6164E-05_r8, 0.7486E-05_r8, 0.9119E-05_r8,            
     &      0.1104E-04_r8,                                                         
     &      0.1221E-04_r8, 0.1246E-04_r8, 0.1203E-04_r8, 0.1084E-04_r8,            
     &      0.9583E-05_r8,                                                         
     &      0.8950E-05_r8, 0.8734E-05_r8, 0.8511E-05_r8, 0.7807E-05_r8,            
     &      0.6639E-05_r8,                                                         
     &      0.5278E-05_r8, 0.3853E-05_r8, 0.2741E-05_r8, 0.2055E-05_r8,            
     &      0.1633E-05_r8,                                                         
     &      0.1216E-05_r8, 0.8718E-06_r8, 0.6456E-06_r8, 0.4707E-06_r8,            
     &      0.3398E-06_r8,                                                         
     &      0.2414E-06_r8, 0.1768E-06_r8, 0.1446E-06_r8, 0.1229E-06_r8,            
     &      0.1035E-06_r8,                                                         
     &      0.8560E-07_r8, 0.7366E-07_r8, 0.6869E-07_r8, 0.6504E-07_r8,            
     &      0.6176E-07_r8,                                                         
     &      0.5887E-07_r8, 0.5607E-07_r8, 0.5348E-07_r8, 0.5117E-07_r8,            
     &      0.4898E-07_r8,                                                         
     &      0.4683E-07_r8, 0.4489E-07_r8, 0.4316E-07_r8, 0.4146E-07_r8,            
     &      0.3979E-07_r8,                                                         
     &      0.3845E-07_r8, 0.3745E-07_r8, 0.3650E-07_r8, 0.3554E-07_r8,            
     &      0.3460E-07_r8,                                                         
     &      0.3369E-07_r8, 0.3282E-07_r8, 0.3196E-07_r8, 0.3111E-07_r8,            
     &      0.3028E-07_r8/
        '''
        

def s_trp75(var):
    if var == 'p':
        return '''
       data (plevel(i), i = 1, nlayer + 1)/                                        
     &      0.0000000_r8, 0.0006244_r8, 0.0008759_r8, 0.0012286_r8,                
     &      0.0017234_r8,                                                          
     &      0.0024174_r8, 0.0033909_r8, 0.0047565_r8, 0.0066720_r8,                
     &      0.0093589_r8,                                                          
     &      0.0131278_r8, 0.0184145_r8, 0.0258302_r8, 0.0362323_r8,                
     &      0.0508234_r8,                                                          
     &      0.0712906_r8, 0.1000000_r8, 0.1402710_r8, 0.1967600_r8,                
     &      0.2759970_r8,                                                          
     &      0.3871_r8, 0.5431_r8, 0.7617_r8, 1.0685_r8, 1.4988_r8,                 
     &      2.1024_r8,                                                             
     &      2.9490_r8, 4.1366_r8, 5.8025_r8, 8.1392_r8, 11.4170_r8,                
     &      16.0147_r8,                                                            
     &      22.4640_r8, 31.5105_r8, 44.2001_r8, 62.0000_r8, 85.7750_r8,            
     &      109.5500_r8,                                                           
     &      133.3250_r8, 157.1000_r8, 180.8750_r8, 204.6500_r8,                    
     &      228.4250_r8, 252.2000_r8,                                              
     &      275.9750_r8, 299.7500_r8, 323.5250_r8, 347.3000_r8,                    
     &      371.0750_r8, 394.8500_r8,                                              
     &      418.6250_r8, 442.4000_r8, 466.1750_r8, 489.9500_r8,                    
     &      513.7250_r8, 537.5000_r8,                                              
     &      561.2750_r8, 585.0500_r8, 608.8250_r8, 632.6000_r8,                    
     &      656.3750_r8, 680.1500_r8,                                              
     &      703.9250_r8, 727.7000_r8, 751.4750_r8, 775.2500_r8,                    
     &      799.0250_r8, 822.8000_r8,                                              
     &      846.5750_r8, 870.3500_r8, 894.1250_r8, 917.9000_r8,                    
     &      941.6750_r8, 965.4500_r8,                                              
     &      989.2250_r8, 1013.0000_r8/
        '''
    elif var == 't':
        return '''
       data (tlayer(i), i = 1, nlayer)/                                            
     &      210.2409_r8, 210.5239_r8, 211.0985_r8, 211.6824_r8,                    
     &      212.2677_r8, 212.8542_r8,                                              
     &      213.4418_r8, 214.0308_r8, 214.6209_r8, 215.2122_r8,                    
     &      215.8047_r8, 216.3985_r8,                                              
     &      216.9957_r8, 217.7589_r8, 220.4488_r8, 225.8594_r8,                    
     &      232.3394_r8, 239.0249_r8,                                              
     &      245.8975_r8, 252.9619_r8, 260.2183_r8, 266.9444_r8,                    
     &      269.2215_r8, 267.0145_r8,                                              
     &      263.0619_r8, 257.6949_r8, 252.0602_r8, 246.5490_r8,                    
     &      241.1625_r8, 235.8975_r8,                                              
     &      230.7513_r8, 225.7210_r8, 220.8037_r8, 215.8554_r8,                    
     &      209.3228_r8, 201.4490_r8,                                              
     &      196.9717_r8, 200.1833_r8, 207.3744_r8, 213.7087_r8,                    
     &      219.3563_r8, 224.4651_r8,                                              
     &      229.1386_r8, 233.4520_r8, 237.4624_r8, 241.2137_r8,                    
     &      244.7407_r8, 248.0714_r8,                                              
     &      251.2287_r8, 254.2318_r8, 257.0965_r8, 259.8364_r8,                    
     &      262.4629_r8, 264.9861_r8,                                              
     &      267.4148_r8, 269.7563_r8, 272.0171_r8, 274.2024_r8,                    
     &      276.3143_r8, 278.3489_r8,                                              
     &      280.2858_r8, 282.0777_r8, 283.6658_r8, 285.0422_r8,                    
     &      286.2809_r8, 287.4879_r8,                                              
     &      288.7410_r8, 290.0624_r8, 291.4273_r8, 292.8017_r8,                    
     &      294.1639_r8, 295.5044_r8,                                              
     &      296.8197_r8, 298.1095_r8, 299.3740_r8/
        '''
    elif var == 'h2o':
        return '''
       data (wlayer(i), i = 1, nlayer)/                                            
     &      0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8,            
     &      0.3250E-05_r8,                                                         
     &      0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8,            
     &      0.3250E-05_r8,                                                         
     &      0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8,            
     &      0.3250E-05_r8,                                                         
     &      0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8,            
     &      0.3250E-05_r8,                                                         
     &      0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8,            
     &      0.3250E-05_r8,                                                         
     &      0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8,            
     &      0.3250E-05_r8,                                                         
     &      0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8, 0.3250E-05_r8,            
     &      0.3250E-05_r8,                                                         
     &      0.3250E-05_r8, 0.3250E-05_r8, 0.3254E-05_r8, 0.3379E-05_r8,            
     &      0.3885E-05_r8,                                                         
     &      0.5967E-05_r8, 0.1417E-04_r8, 0.2944E-04_r8, 0.5517E-04_r8,            
     &      0.9524E-04_r8,                                                         
     &      0.1538E-03_r8, 0.2351E-03_r8, 0.3388E-03_r8, 0.4425E-03_r8,            
     &      0.5666E-03_r8,                                                         
     &      0.7130E-03_r8, 0.8837E-03_r8, 0.1081E-02_r8, 0.1305E-02_r8,            
     &      0.1560E-02_r8,                                                         
     &      0.1846E-02_r8, 0.2165E-02_r8, 0.2520E-02_r8, 0.2910E-02_r8,            
     &      0.3339E-02_r8,                                                         
     &      0.3807E-02_r8, 0.4314E-02_r8, 0.4882E-02_r8, 0.5916E-02_r8,            
     &      0.7010E-02_r8,                                                         
     &      0.8139E-02_r8, 0.9275E-02_r8, 0.9960E-02_r8, 0.1054E-01_r8,            
     &      0.1116E-01_r8,                                                         
     &      0.1184E-01_r8, 0.1258E-01_r8, 0.1338E-01_r8, 0.1423E-01_r8,            
     &      0.1516E-01_r8/
        '''
    elif var == 'o3':
        return '''
       data (olayer(i), i = 1, nlayer)/                                            
     &      0.1993E-06_r8, 0.2257E-06_r8, 0.2786E-06_r8, 0.3318E-06_r8,            
     &      0.3853E-06_r8,                                                         
     &      0.4391E-06_r8, 0.4932E-06_r8, 0.5476E-06_r8, 0.6022E-06_r8,            
     &      0.6572E-06_r8,                                                         
     &      0.7125E-06_r8, 0.7680E-06_r8, 0.8239E-06_r8, 0.8807E-06_r8,            
     &      0.1017E-05_r8,                                                         
     &      0.1288E-05_r8, 0.1622E-05_r8, 0.1975E-05_r8, 0.2347E-05_r8,            
     &      0.2739E-05_r8,                                                         
     &      0.3151E-05_r8, 0.3574E-05_r8, 0.4269E-05_r8, 0.5379E-05_r8,            
     &      0.6905E-05_r8,                                                         
     &      0.8719E-05_r8, 0.9931E-05_r8, 0.1045E-04_r8, 0.1122E-04_r8,            
     &      0.1234E-04_r8,                                                         
     &      0.1212E-04_r8, 0.1028E-04_r8, 0.7945E-05_r8, 0.5155E-05_r8,            
     &      0.2594E-05_r8,                                                         
     &      0.1040E-05_r8, 0.3939E-06_r8, 0.2292E-06_r8, 0.1898E-06_r8,            
     &      0.1641E-06_r8,                                                         
     &      0.1454E-06_r8, 0.1282E-06_r8, 0.1139E-06_r8, 0.1023E-06_r8,            
     &      0.9333E-07_r8,                                                         
     &      0.8689E-07_r8, 0.8190E-07_r8, 0.7752E-07_r8, 0.7416E-07_r8,            
     &      0.7200E-07_r8,                                                         
     &      0.7025E-07_r8, 0.6863E-07_r8, 0.6710E-07_r8, 0.6568E-07_r8,            
     &      0.6435E-07_r8,                                                         
     &      0.6307E-07_r8, 0.6188E-07_r8, 0.6077E-07_r8, 0.5968E-07_r8,            
     &      0.5899E-07_r8,                                                         
     &      0.5869E-07_r8, 0.5839E-07_r8, 0.5798E-07_r8, 0.5737E-07_r8,            
     &      0.5669E-07_r8,                                                         
     &      0.5602E-07_r8, 0.5531E-07_r8, 0.5452E-07_r8, 0.5374E-07_r8,            
     &      0.5297E-07_r8,                                                         
     &      0.5213E-07_r8, 0.5115E-07_r8, 0.5012E-07_r8, 0.4910E-07_r8,            
     &      0.4810E-07_r8/
        '''
    


In [36]:
'''
Load atmosphere profile data 
'''

def fortran2array(s):
    return np.array(numbers(s)).astype(float).reshape(-1, 1)


def convert2dataframe(func):
    
    def _func(*args, **kwargs):
        p, t, h2o, o3 = func(*args, **kwargs)
        p = p[2:]
        t = t[1:]
        h2o = h2o[1:]
        o3 = o3[1:]
        p = .5 * (p[1:] + p[:-1])
        plog = np.log(p)
    
        df = pd.DataFrame({'t': t.reshape(-1,),
                           'p': p.reshape(-1,),
                           'logp': plog.reshape(-1,),
                           'h2o': h2o.reshape(-1,),
                           'o3': o3.reshape(-1,)})

        df = df.set_index(['logp'])
        return df
    return _func


@convert2dataframe
def load_crd_profile(atmpro='mls'):
    '''
    Loads plevel and tlayer from .pro files used in CRD.
    
    Parameters
    ----------
    atmpro: string
        'mls', 'saw', or 'trp'
    t: numpy.array
        layer temperature
    p: numpy.array
        level pressure
    '''
    if atmpro == 'mls':
        func = s_mls75
    elif atmpro == 'saw':
        func = s_saw75
    elif atmpro == 'trp':
        func = s_trp75
    
    p = fortran2array(func(var='p'))    
    t = fortran2array(func(var='t'))
    h2o = fortran2array(func(var='h2o'))
    o3 = fortran2array(func(var='o3'))
    return p, t, h2o, o3

In [37]:
'''
Write out atmosphere profile files that are compatible with clirad.
'''

def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4, dtype=float):
    '''
    '''
    if dtype == float:
        strfmt = '{:15.6e}'
    elif dtype == int:
        strfmt = '{:15d}'
    else:
        raise ValueError('dtype must be either float or int.')
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines


def vector_to_F77(array=None, num_values_per_line=None, dtype=None):
    lines = vector_to_F77list(array=array, 
                              num_values_per_line=num_values_per_line,
                              dtype=dtype)
    
    rlines = [5 * ' ' + '&' + l for l in lines]
    
    fortran = '\n'.join(rlines)
    return fortran



################


def atmpro_level_var(vs, name):
    s = vector_to_F77(vs, num_values_per_line=3, dtype=float)
    ls = ['data ({}(1, i), i = 1, np + 1) /'.format(name),
          s,
          5 * ' ' + '&' + '/']
    return '\n'.join(ls)


def atmpro_layer_var(vs, name):
    s = vector_to_F77(vs, num_values_per_line=3, dtype=float)
    ls = ['data ({}(1, i), i = 1, np) /'.format(name),
          s,
          5 * ' ' + '&' + '/']
    return '\n'.join(ls)


def file_content(atmpro='mls'):
    name_map = {'p': 'pl', 't': 'ta', 'h2o': 'wa', 'o3': 'oa'}
    
    df = load_crd_profile(atmpro=atmpro)
    
    lines = []
    for var in name_map.keys():
        if var == 'p':
            func = atmpro_level_var
        else:
            func = atmpro_layer_var
            
        s = func(df[var], name_map[var])
        lines.append(s)
    
    lines = [6 * ' ' + l for l in lines]
    s = '\n'.join(lines)
    return s
        

def script():
    dir_atmpro = os.path.join('/chia_cluster/home/jackyu',
                              'radiation/clirad-lw/LW',
                              'lee_hitran2012_update',
                              'atmosphere_profiles')
    
    for atmpro in ('mls', 'saw', 'trp'):
        fpath = os.path.join(dir_atmpro,
                             '{}75.pro'.format(atmpro))
        s = file_content(atmpro=atmpro)
        with open(fpath, mode='w', encoding='utf-8') as f:
            f.write(s)
            
            

In [38]:
script()

In [13]:
def plt_profile(var='t'):
    titles = {'t': 'temperature',
              'h2o': 'H2O concentration',
              'o3': 'O3 concentration'}
    xlabels = {'t': 'temperature [K]',
               'h2o': 'mass-mixing ratio [g/g]',
               'o3': 'mass-mixing ratio [g/g]'}
    
    atmpro_mls = load_crd_profile(atmpro='mls')
    atmpro_saw = load_crd_profile(atmpro='saw')
    atmpro_trp = load_crd_profile(atmpro='trp')

    p = figure(x_axis_type='log', y_axis_type='log',
               title=titles[var])

    p.cross(atmpro_mls[var], atmpro_mls['p'], 
            legend='mid-latitude summer',
            color='green', alpha=.5, size=12)
    p.circle(atmpro_saw[var], atmpro_saw['p'], 
             legend='sub-arctic winter',
             color='navy', alpha=.3, size=7)
    p.circle(atmpro_trp[var], atmpro_trp['p'], 
             legend='tropical',
             color='red', alpha=.5, size=7)

    p.y_range = Range1d(atmpro_mls['p'].max(), atmpro_mls['p'].min())
    p.yaxis.axis_label = 'pressure [mb]'
    p.xaxis.axis_label = xlabels[var]

    show(p)

In [14]:
plt_profile(var='t')

In [15]:
plt_profile(var='h2o')

In [16]:
plt_profile(var='o3')

In [17]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')